In [2]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)

In [13]:
import keras
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Dense, Activation, Input, MaxPooling2D, Flatten
from keras.layers import Conv2D

In [9]:
batch_size = 32
num_classes = 10
epochs = 5

In [10]:
#The data shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print('x_train shape: ', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape:  (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [11]:
#Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Inception Module

Take a look at the inception module figure here: https://arxiv.org/pdf/1409.4842.pdf

In [14]:
#Input layer is the same as our typical CNN model
inputs = Input(shape=(32,32,3))

##----------------------New stuffs starts here---------------------

tower_1 = Conv2D(64, (1,1), padding='same', activation='relu')(inputs)
tower_1 = Conv2D(64, (3,3), padding='same', activation='relu')(tower_1)


tower_2 = Conv2D(64, (1,1), padding='same', activation='relu')(inputs)
tower_2 = Conv2D(64, (5,5), padding='same', activation='relu')(tower_2)


tower_3 = MaxPooling2D((3,3), padding='same', strides=(1,1))(inputs)
tower_3 = Conv2D(64, (1,1), padding='same', activation='relu')(tower_3)

x = keras.layers.concatenate([tower_1, tower_2, tower_3], axis = 1)

##----------------------New stuffs ends here-----------------------

x = Conv2D(8, (3,3))(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Flatten()(x)

x = Dense(num_classes)(x)

output = Activation('softmax')(x)

In [15]:
model = Model([inputs], output)

In [16]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 32, 32, 64)   256         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 32, 32, 64)   256         input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 32, 32, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_6 (

In [ ]:
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

#Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, batch_size= batch_size, epochs= epochs, validation_data= (x_test, y_test))

In [ ]:
#Scored trained model.

scores = model.evaluate(x_test, y_test, verbose = 1)
print('Test loss: ', scores[0])
print('Test accuracy: ', scores[1])